# Part IIA: Explicitly solving Tic-Tac-Toe with the Minimax Algorithm

- To know more about the Minimax algorithm, you can read [the notes I have prepared](../report.pdf) or the [Wikipedia article on it](https://en.wikipedia.org/wiki/Minimax). It is a very general tool for solving decision-making problems, however, it can become very computationally expensive. We will analyze this algorithm for the special case of finite terminating two player games.

- Tic-Tac-Toe is a finite terminating two player game that can only end in a loss for one player (and a win for the other) or in a tie. It can be shown that for every such game, at least one of the players has a non-losing strategy (some of you may see this proof again later in CS 207 - it's basically structural induction over Game Trees). This includes games such as Chess!

- The algorithm is based on forming the game tree: The root of this tree is the starting position, and the children of each node are the positions reachable from the node position. If the game is lost/won at a position, it is terminated, ie, the corresponding node has no children.

- Nodes at even levels (the root is at level 0) are the positions when the first player (call her A) is to play and nodes at the odd levels are the positions when the second player (call her B) is to play.

- Now, we say a node is winning iff there exists a strategy for the player of that node such that no matter what the other player does, she will win. A node is losing iff there if no matter what the player of the node does, the other player can play in such a way that she loses. If neither are true, we say the node is drawn.

- Now, if a node is winning, then there must exist a move the node player makes such that from then on the other player is losing, even if she plays perfectly. A move is a transition from a parent node to a child node (remember parent player and child player are different).

- - So, a node is winning if there exists a child node that is losing.
- - A node is drawn if there exists no losing child node but there does exist a drawn child node.
- - A node is losing if every child node is winning.

- Since we know the statuses of all the leaf nodes (all either winning, losing or drawn), we can get the statuses of every node (ie every position) from the bottom up, recursively. Hence, every node is either winning, losing or drawn, including the root node.

- This proves the statement above, and this algorithm for finding the node statuses is known as the Minimax algorithm.

- The disadvantage is that this algorithm is extremely computationally expensive. It's training time (time taken to find out status of each node) is proportional to the number of nodes, ie, the number of reachable game states. For a game like Chess, this is on the order of 10^40!

- Therefore, the minimax algorithm is not really used, except for tiny problems, and instead, real reinforcement learning algorithms are used, which may be less accurate, but are way less computationally expensive.

In [5]:
class TicTacToeEngine:

    def is_full(self, board):
        n = len(board)
        for row in board:
            if '-' in row:
                return False
        return True
    
    def evaluate(self, board):
        n = len(board)
        for row in board:
            if row.count(row[0]) == n and row[0] != '-':
                return 1 if row[0] == 'X' else -1

        for col in range(n):
            column = [board[i][col] for i in range(n)]
            if column.count(column[0]) == n and column[0] != '-':
                return 1 if column[0] == 'X' else -1

        diag1 = [board[i][i] for i in range(n)]
        if diag1.count(diag1[0]) == n and diag1[0] != '-':
            return 1 if diag1[0] == 'X' else -1

        diag2 = [board[i][n-i-1] for i in range(n)]
        if diag2.count(diag2[0]) == n and diag2[0] != '-':
            return 1 if diag2[0] == 'X' else -1

        return 0

    def minimax(self, gboard, depth, is_maximizing):
        board = [i for i in gboard]
        score = self.evaluate(board)
        if score == 1:
            return score - depth
        if score == -1:
            return score + depth
        if self.is_full(board):
            return 0

        if is_maximizing:
            best_score = -9999999
            for row in range(len(board)):
                for col in range(len(board[row])):  
                    if board[row][col] == '-':
                        board[row][col] = 'X'
                        score = self.minimax(board, depth + 1, False)
                        board[row][col] = '-'
                        best_score = max(score, best_score)
            return best_score
        else:
            best_score = 9999999
            for row in range(len(board)):
                for col in range(len(board[row])):
                    if board[row][col] == '-':
                        board[row][col] = 'O'
                        score = self.minimax(board, depth + 1, True)
                        board[row][col] = '-'
                        best_score = min(score, best_score)
            return best_score


    def find_best_move(self, board):
        best_score = -9999999
        best_move = None
        for row in range(len(board)):
            for col in range(len(board[row])):
                if board[row][col] == '-':
                    board[row][col] = 'X'
                    score = self.minimax(board, 0, False)
                    board[row][col] = '-'
                    if score > best_score:
                        best_score = score
                        best_move = (row, col)
        return best_move
    

    def print_board(self, board):
        n = len(board)
        for row in board:
            print(' '.join(row))
        print()



    def play(self, n):
        board = [['-' for _ in range(n)] for _ in range(n)]
        current_player = 'X'
        while True:
            self.print_board(board)
            if current_player == 'X':
                row, col = self.find_best_move(board)
                board[row][col] = current_player
                print(f"Player {current_player} plays at position ({row}, {col})")
            else:
                valid_move = False
                while not valid_move:
                    try:
                        row = int(input("Enter row: "))
                        col = int(input("Enter column: "))
                        if board[row][col] == '-':
                            valid_move = True
                        else:
                            print("Invalid move. Try again.")
                    except (ValueError, IndexError):
                        print("Invalid input. Try again.")
                board[row][col] = current_player

            if self.evaluate(board) != 0 or self.is_full(board):
                break

            current_player = 'O' if current_player == 'X' else 'X'

        self.print_board(board)
        score = self.evaluate(board)
        if score == 1:
            print("Player X wins!")
        elif score == -1:
            print("Player O wins!")
        else:
            print("It's a tie!")

In [9]:
game = TicTacToeEngine()
game.play(3)

- - -
- - -
- - -

Player X plays at position (1, 1)
- - -
- X -
- - -

- - O
- X -
- - -

Player X plays at position (2, 0)
- - O
- X -
X - -

O - O
- X -
X - -

Player X plays at position (1, 0)
O - O
X X -
X - -

O - O
X X O
X - -

Player X plays at position (0, 1)
O X O
X X O
X - -

Invalid move. Try again.
O X O
X X O
X O -

Player X plays at position (2, 2)
O X O
X X O
X O X

It's a tie!
